#Image to Image Search Engine

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
import os
from zipfile import ZipFile

In [ ]:
# Mount Google Drive if needed
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# Download and extract dataset
!pip install kaggle
!mkdir -p ~/.kaggle
!cp /content/drive/My\ Drive/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d vikashrajluhaniwal/fashion-images
!unzip fashion-images.zip -d fashion-images


cp: cannot stat '/content/drive/My Drive/kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/vikashrajluhaniwal/fashion-images
License(s): CC0-1.0
 99% 330M/335M [00:05<00:00, 57.1MB/s]
100% 335M/335M [00:05<00:00, 59.8MB/s]
Archive:  fashion-images.zip
  inflating: fashion-images/data/Apparel/Boys/Images/images_with_product_ids/10054.jpg  
  inflating: fashion-images/data/Apparel/Boys/Images/images_with_product_ids/10649.jpg  
  inflating: fashion-images/data/Apparel/Boys/Images/images_with_product_ids/10671.jpg  
  inflating: fashion-images/data/Apparel/Boys/Images/images_with_product_ids/12840.jpg  
  inflating: fashion-images/data/Apparel/Boys/Images/images_with_product_ids/12844.jpg  
  inflating: fashion-images/data/Apparel/Boys/Images/images_with_product_ids/12845.jpg  
  inflating: fashion-images/data/Apparel/Boys/Images/images_with_product_ids/12846.jpg  
  inflating: f

In [ ]:
# Initialize VGG16 model for feature extraction
model = VGG16(weights='imagenet', include_top=False)
model.summary()

58889256/58889256 [==============================] - 1s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 block1_conv1 (Conv2D)       (None, None, None, 64)    1792      
                                                                 
 block1_conv2 (Conv2D)       (None, None, None, 64)    36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, None, None, 64)    0         
                                                                 
 block2_conv1 (Conv2D)       (None, None, None, 128)   73856     
                                                                 
 block2_conv2 (Conv2D)       (None, None, None, 128)   147584    
                                                              

#Feature Exctraction

In [ ]:
# Function to preprocess and extract features
def extract_features(img_path, model):
    img = image.load_img(img_path, target_size=(224, 224))
    img_data = image.img_to_array(img)
    img_data = np.expand_dims(img_data, axis=0)
    img_data = preprocess_input(img_data)

    features = model.predict(img_data)
    return features.flatten()

In [ ]:
# Function to preprocess and extract features
def extract_features(img_path, model):
    try:
        img = image.load_img(img_path, target_size=(224, 224))
        img_data = image.img_to_array(img)
        img_data = np.expand_dims(img_data, axis=0)
        img_data = preprocess_input(img_data)

        features = model.predict(img_data)
        return features.flatten()
    except (UnidentifiedImageError, OSError) as e:
        print(f"Error processing image {img_path}: {e}")
        return None

# Extract features for all images in the dataset
image_folder = '/content/fashion-images/data'
image_paths = [os.path.join(image_folder, img) for img in os.listdir(image_folder) if img.endswith(('.jpg', '.jpeg', '.png'))] # Filter for common image extensions
features_list = [extract_features(img_path, model) for img_path in image_paths]

# Remove None values (failed image extractions) and convert to NumPy array
features = np.array([f for f in features_list if f is not None])

In [ ]:
# Save features and image paths for later use
np.save('features.npy', features)
np.save('image_paths.npy', image_paths)

In [ ]:
# Load features and image paths
features = np.load('features.npy')
image_paths = np.load('image_paths.npy')

# Check if features array is empty and handle accordingly
if features.size == 0:
    print("Error: No features extracted. Check image path and feature extraction process.")
else:
    # Reshape features to 2D if necessary
    if features.ndim == 1:
        features = features.reshape(-1, 1)  # Reshape for single feature

    # Fit Nearest Neighbors model
    neighbors = NearestNeighbors(n_neighbors=5, algorithm='ball_tree')
    neighbors.fit(features)

##Find Similar Images

In [ ]:
# Function to find similar images
def find_similar_images(query_image_path, model, neighbors, image_paths, top_n=5):
    query_features = extract_features(query_image_path, model)
    distances, indices = neighbors.kneighbors([query_features])

    similar_images = [image_paths[idx] for idx in indices[0][:top_n]]
    return similar_images


##Image Search Engine

In [ ]:
def image_search(query_img):
    similar_images = find_similar_images(query_img.name, model, neighbors, image_paths, top_n=5)
    return similar_images


#Fashion Designing System

In [ ]:
from PIL import Image
import requests
import tempfile


##Design Evaluation

In [ ]:
def evaluate_design(input_img):
    # Process the uploaded image
    img = Image.open(input_img.name)
    # Perform evaluation or modification with your language model
    # Example:
    # transformed_image = your_model_function(img)
    transformed_image = img  # Placeholder for actual processing
    return transformed_image


In [ ]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 857.8/857.8 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.1/318.1 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 7.4 MB/s

##Review Design

In [ ]:
import gradio as gr

# Create Gradio interfaces
image_search_interface = gr.Interface(
    fn=image_search,
    inputs=gr.Image(type="filepath", label="Upload Image"),  # Change type to "filepath"
    outputs=gr.Image(label="Similar Images"),
    title="Image-to-Image Search Engine",
    description="Upload an image to find visually similar images from the dataset."
)

design_review_interface = gr.Interface(
    fn=evaluate_design,
    inputs=gr.Image(type="filepath", label="Upload Design"),   # Change type to "filepath"
    outputs=gr.Image(label="Reviewed Design"),
    title="Fashion Design Review",
    description="Upload your fashion design to receive feedback or enhancements."
)

# Launch the Gradio interfaces
# Provide inputs and outputs for the combined interface
demo = gr.Interface(
    fn=lambda x: [image_search(x), evaluate_design(x)],  # Combine the functions
    inputs=gr.Image(type="filepath", label="Upload Image"),
    outputs=[gr.Image(label="Similar Images"), gr.Image(label="Reviewed Design")],
    title="Image Search and Design Review",
    description="Upload an image to search for similar images and get design feedback."
)

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://40b9d875cf0797c58e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
